<a href="https://colab.research.google.com/github/kunalpagarey/PyTocrh_Tutorial/blob/master/PyTorch_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [0]:
transform = transforms.Compose([transforms.Resize((28, 28)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.MNIST('./data/', train=True, transform=transform, download=True)
val_data = datasets.MNIST('./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
val_loader = DataLoader(val_data, batch_size=100)

def convert_image(img):
    image = img.cpu().clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
    image = image.clip(0, 1)
    return image


In [0]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(labels)
fig = plt.figure(figsize=(25, 4))

for idx in np.arange(20):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    plt.imshow(convert_image(images[idx]))
    ax.set_title([labels[idx].item()])

In [0]:
# LeNet implementation

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
    
        return x

In [0]:
model = LeNet().to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [0]:
epochs = 15
t_losses = []
t_accuracy = []
v_losses = []
v_accuracy = []
for e in range(epochs):
    te_loss = 0.0
    te_correct = 0.0
    ve_loss = 0.0
    ve_correct = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, pred = torch.max(outputs, 1)

        te_loss += loss.item()
        te_correct += torch.sum(pred == labels)

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            _, pred = torch.max(outputs, 1)

            ve_loss += loss.item()
            ve_correct += torch.sum(pred == labels)
    
    te_loss = te_loss/len(train_loader)
    te_correct = te_correct/len(train_loader)
    ve_loss = ve_loss/len(val_loader)
    ve_correct = ve_correct/len(val_loader)

    t_losses.append(te_loss)
    t_accuracy.append(te_correct)
    v_losses.append(ve_loss)
    v_accuracy.append(ve_correct)
    print("Epoch: {}, Train: Loss={:0.4f}, Accuracy={:0.4f}, Val: Loss={:0.4f}"\
          ", Accuracy={:0.4f}".format(e, te_loss, te_correct.item(), ve_loss,
                                      ve_correct.item()))        

In [0]:
plt.plot(t_losses, label="Training losses")
plt.plot(v_losses, label="Validation losses")
plt.title("Loss curves.")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [0]:
plt.plot(t_accuracy, label="Training accuracy")
plt.plot(v_accuracy, label="Validation accuracy")
plt.title("Accuracy curves.")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [0]:
import PIL.ImageOps as ops
from PIL import Image
import requests

In [0]:
url = 'https://images.homedepot-static.com/productImages/007164ea-d47e-4f66-8d8c-fd9f621984a2/svn/architectural-mailboxes-house-letters-numbers-3585b-5-64_1000.jpg'

response = requests.get(url, stream=True)
img = Image.open(response.raw)
img = ops.invert(img)
img=img.convert('1')
img = transform(img)
plt.imshow(convert_image(img))

In [0]:
im = img.to(device)
im = im[0].unsqueeze(0).unsqueeze(0)
out = model(im)
_, prediction = torch.max(out, 1)
print(prediction.item())

In [0]:
dataiter = iter(val_loader)
images, labels = dataiter.next()
images = images.to(device)
labels = labels.to(device)
logits = model(images)
_, predictions = torch.max(logits, 1)

fig = plt.figure(figsize=(25, 4))

for idx in np.arange(20):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    plt.imshow(convert_image(images[idx]))
    ax.set_title("{}({})".format(str(labels[idx].item()), str(predictions[idx].item())), 
                 color="green" if predictions[idx] == labels[idx] else "red")